In [1]:
import re
import win32com.client 
import os
import sys
import pandas as pd
import csv
import requests 
from pandas.core.frame import DataFrame
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

def email_scrap(title):
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    inbox = outlook.GetDefaultFolder(6).Folders["Folder_A"]
    today = datetime.today()
    received_today = inbox.Items.Restrict("[ReceivedTime] >= '"+ today.strftime('%d/%m/%Y')+"'")
    email_sub = title
    for email in received_today:
        if email.Subject == email_sub:
            print(email.Subject, email.ReceivedTime, email.SenderEmailAddress)
            body_content = email.HTMLBody
            bs = BeautifulSoup(body_content, features="html.parser")
            tables = bs.find_all("table")
            table = tables[0] 
            
            rows = table.find_all("tr")
            table_data = []
            for row in rows:
                row_data = []
                cells = row.find_all(['td', 'th'])
                for cell in cells:
                    cell_data = cell.get_text().strip()
                    row_data.append(cell_data)
                table_data.append(row_data)
            headers = table_data.pop(0)
            df = pd.DataFrame(table_data, columns=headers)
            df.columns.values[0] = "Number"
            break
    
    else:
        print("No table to be scrape")
    return df
